In [ ]:
@info "Imports ..."
using Revise
using ConvexHullPricing
using DataFrames
using SparseArrays
using Plots
using JLD2
using LinearAlgebra, LaTeXStrings
using ProgressBars
const UT = ConvexHullPricing.Utilitaries
const OPT = ConvexHullPricing.Optimizer
BEinstances = []
for file in readdir("..//data//belgian"; join=true)
    push!(BEinstances, UT.load_data(file))
end
CAinstances = []
for file in readdir("..//data//ca"; join=true)
    push!(CAinstances, UT.load_data(file))
end

function make_it_monotone(array)
    new_array = Float64[]
    push!(new_array, array[1])
    for elt in array[2:end]
        if elt < minimum(new_array)
            push!(new_array, elt)
        else
            push!(new_array, minimum(new_array))
        end
    end
    return new_array
end


N_MIN = 5
τ = N_MIN * 60
idx = 1
@info "Instance $idx"
Belgian = BEinstances[idx]
X0 = UT.LP_Relaxation(Belgian)
FS = maximum(load_object("december//UltraOptimalRunsBE.jld2")[idx][3])
# Compilation
_, _, Hval, _ = OPT.tShiftedFGM(Belgian, X0, 1., 0.1, 1e-8)
_, _, Hval, _ = OPT.tGM(Belgian, X0, 1., 0.5, 1e-8)


In [ ]:

function logrange(p::Pair{<:Real, <:Real}, n::Integer)
	lo, hi = promote(p.first, p.second)
	invert = abs(lo) > abs(hi)
	ratio = invert ? lo/hi : hi/lo
	if ratio % 10 == 0
		exp = exp10
		log = log10
	elseif ratio % 2 == 0
		exp = exp2
		log = log2
	else
		exp = Base.exp
		log = Base.log
	end
	if ratio > 0
		if invert
			log_ratio = -log(ratio)
			(x > log_ratio/2 ? lo*exp(x) : hi*exp(x-log_ratio) for x in range(0, log_ratio, length=n))
		else
			log_ratio = log(ratio)
			(x < log_ratio/2 ? lo*exp(x) : hi*exp(x-log_ratio) for x in range(0, log_ratio, length=n))
		end
	else
		throw(DomainError(p, "logrange requires that the first and last elements are both positive, or both negative"))
	end
end

RanLambda = [k for k in logrange(1e-4 => 0.008, 20)]
GM_vals = Float64[]
FGM_vals = Float64[]

In [ ]:
RanLambda

In [ ]:
xax, yax = load_object("GM_tuning_on_Autumn_WD")

In [ ]:
@info xax[argmin(yax)]

In [ ]:
#= for λ in RanLambda
	@info "λ = $λ ; GD run -->"
	_, Hit, Hval, _ = OPT.tGM(Belgian, X0, τ, λ, 1e-8)
	Htrue = Float64[]
	@info "done"
	for price in Hit
		push!(Htrue, UT.exact_oracle(Belgian, price)[1])
	end
	push!(GM_vals, minimum(FS .- Htrue))
end
save_object("GM_tuning_on_Autumn_WD", [RanLambda, GM_vals])
=#
for λ in RanLambda
	@info "λ = $λ ; FGM run -->"
	_, Hit, Hval, _ = OPT.tShiftedFGM(Belgian, X0, τ, λ, 1e-8)
	Htrue = Float64[]
	@info "done"
	for price in Hit
		push!(Htrue, UT.exact_oracle(Belgian, price)[1])
	end
	push!(FGM_vals, minimum(FS .- Htrue))
end
save_object("FGM_tuning_on_Autumn_WD", [RanLambda, FGM_vals])

In [ ]:
save_object("FGM_tuning_on_Autumn_WD", [RanLambda, FGM_vals])